In [7]:
# Import the necessary libraries
import pandas as pd
from datetime import datetime as dt
import time
import warnings

# Show all columns
pd.set_option('display.max_columns', None)

# Disable wraning
warnings.filterwarnings('ignore')

In [4]:
# Source data URL link from - https://www.redfin.com/news/data-center/
housing_by_city = 'https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_market_tracker/city_market_tracker.tsv000.gz'

# The function that would extract the source data
def extract_data(url):
    df = pd.read_csv(url, compression='gzip', sep='\t')
    return df

In [9]:
# Call the Extract function and pass the URL (this takes about 3 minutes to run)
df = extract_data(housing_by_city)

In [10]:
# View the first 5 records
df.head()

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,state_code,property_type,property_type_id,median_sale_price,median_sale_price_mom,median_sale_price_yoy,median_list_price,median_list_price_mom,median_list_price_yoy,median_ppsf,median_ppsf_mom,median_ppsf_yoy,median_list_ppsf,median_list_ppsf_mom,median_list_ppsf_yoy,homes_sold,homes_sold_mom,homes_sold_yoy,pending_sales,pending_sales_mom,pending_sales_yoy,new_listings,new_listings_mom,new_listings_yoy,inventory,inventory_mom,inventory_yoy,months_of_supply,months_of_supply_mom,months_of_supply_yoy,median_dom,median_dom_mom,median_dom_yoy,avg_sale_to_list,avg_sale_to_list_mom,avg_sale_to_list_yoy,sold_above_list,sold_above_list_mom,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
0,2016-09-01,2016-09-30,30,place,6,19944,f,"Waukegan, IL",Waukegan,Illinois,IL,Multi-Family (2-4 Unit),4,100000.0,-0.312715,1.105263,130800.0,0.137391,0.308654,47.206204,-0.140929,0.918270,95.076586,0.131435,0.278040,4.0,-0.333333,0.333333,5.0,-0.166667,-0.166667,5.0,-0.375000,-0.500000,20.0,-0.090909,-0.285714,5.0,1.3,-4.3,55.0,39.0,-228.0,1.111176,0.132120,0.167098,0.250000,-0.083333,0.250000,0.200000,0.109091,0.021429,0.000000,-0.333333,-0.500000,"Lake County, IL",29404,2025-03-10 14:20:36
1,2019-03-01,2019-03-31,30,place,6,18453,f,"Zumbrota, MN",Zumbrota,Minnesota,MN,Single Family Residential,6,233500.0,-0.187543,0.239055,204900.0,0.000000,-0.127714,142.082386,-0.012246,0.315868,94.829222,-0.337494,-0.161620,4.0,3.000000,0.000000,6.0,0.200000,0.200000,3.0,-0.500000,-0.400000,13.0,0.000000,0.083333,3.3,-9.7,0.3,80.0,59.0,59.0,1.006673,0.015297,0.010594,0.250000,0.250000,0.250000,NaN,NaN,NaN,0.500000,0.300000,0.500000,"Red Wing, MN",39860,2025-03-10 14:20:36
2,2015-08-01,2015-08-31,30,place,6,16347,f,"Poquoson, VA",Poquoson,Virginia,VA,Single Family Residential,6,345000.0,0.554054,0.380000,372450.0,0.128807,0.245652,131.699729,-0.085784,0.063953,139.716868,0.106175,0.145872,14.0,0.272727,-0.176471,10.0,-0.333333,0.111111,8.0,-0.666667,-0.652174,79.0,-0.081395,0.025974,5.6,-2.2,1.1,73.0,18.0,-60.0,0.957002,-0.029158,-0.020544,0.142857,0.051948,-0.033613,0.240506,0.031204,-0.032221,0.000000,0.000000,0.000000,"Virginia Beach, VA",47260,2025-03-10 14:20:36
3,2024-01-01,2024-01-31,30,place,6,26370,f,"Whitfield, FL",Whitfield,Florida,FL,All Residential,-1,616250.0,0.354396,0.584190,129900.0,-0.594062,-0.666067,351.688379,0.414483,0.142962,352.348993,0.937919,0.339004,4.0,3.000000,1.000000,4.0,-0.200000,-0.200000,5.0,4.000000,0.666667,21.0,-0.125000,3.200000,5.3,-18.7,2.8,17.0,-82.0,5.0,0.979708,-0.020292,-0.005564,0.000000,0.000000,0.000000,0.285714,0.160714,-0.114286,0.000000,-0.200000,-0.400000,"North Port, FL",35840,2025-03-10 14:20:36
4,2020-10-01,2020-10-31,30,place,6,7685,f,"Hopkins, MN",Hopkins,Minnesota,MN,Single Family Residential,6,330000.0,-0.065156,-0.131579,309450.0,-0.047846,0.289912,202.949073,0.038988,0.151796,177.235722,-0.013925,0.224489,8.0,-0.384615,-0.466667,12.0,0.200000,0.200000,19.0,0.900000,1.714286,19.0,0.461538,0.461538,2.4,1.4,1.5,25.0,13.0,4.0,1.020724,0.008575,0.013794,0.625000,0.009615,0.291667,0.421053,0.190283,-0.040486,0.833333,0.333333,0.533333,"Minneapolis, MN",33460,2025-03-10 14:20:36


In [11]:
# To report the size of the dataset
print('Num of rows:', len(df))
print('Num of cols:', len(df.columns))

Num of rows: 5779946
Num of cols: 58


In [8]:
# Source data URL link from - https://www.redfin.com/news/data-center/
housing_by_city = 'https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_market_tracker/city_market_tracker.tsv000.gz'

# The function that would transform the raw data from source
def extract_transform_data(url):
    df = pd.read_csv(url, compression='gzip', sep='\t')

    # Data cleaning 1: Remove commas from the 'city' column
    df['city'] = df['city'].str.replace(',', '')

    # Data cleaning 2: First select 24 fields from the raw dataset
    cols = ['period_begin','period_end','period_duration', 'region_type', 'region_type_id', 'table_id',
    'is_seasonally_adjusted', 'city', 'state', 'state_code', 'property_type', 'property_type_id',
    'median_sale_price', 'median_list_price', 'median_ppsf', 'median_list_ppsf', 'homes_sold',
    'inventory', 'months_of_supply', 'median_dom', 'avg_sale_to_list', 'sold_above_list',
    'parent_metro_region_metro_code', 'last_updated']
    df = df[cols]

    # Data cleaning 3: Drop all records that have null values
    df = df.dropna()

    # Data cleaning 4: Typecast the 'period_begin' and 'period_end' to datetime datatype
    df['period_begin'] = pd.to_datetime(df['period_begin'])
    df['period_end'] = pd.to_datetime(df['period_end'])

    # Data cleaning 5: Extract year data from 'period_begin_in_years' and 'period_end_in_years' into 2 new columns
    df["period_begin_in_years"] = df['period_begin'].dt.year
    df["period_end_in_years"] = df['period_end'].dt.year

    # Data cleaning 5: Extract month data from 'period_begin_in_months' and 'period_end_in_months' into 2 new columns
    df["period_begin_in_months"] = df['period_begin'].dt.month
    df["period_end_in_months"] = df['period_end'].dt.month

    # Data cleaning 6: Map the month number to their respective month name
    month_dict = {
        "period_begin_in_months": {
            1 : "Jan",
             2 :  "Feb",
             3: "Mar",
             4: "Apr",
             5: "May",
             6: "Jun",
             7: "Jul",
             8: "Aug",
             9: "Sep",
             10: "Oct",
             11: "Nov",
             12: "Dec",
        },
        "period_end_in_months": {
            1 : "Jan",
             2 :  "Feb",
             3: "Mar",
             4: "Apr",
             5: "May",
             6: "Jun",
             7: "Jul",
             8: "Aug",
             9: "Sep",
             10: "Oct",
             11: "Nov",
             12: "Dec"
        }}
    df = df.replace(month_dict)

    # Report the size of the transformed dataset
    print('Num of rows:', len(df))
    print('Num of cols:', len(df.columns))

    # Return the transformed dataset as a dataframe object
    return df

In [9]:
# Call the Extract-Transform function and pass the URL (this takes about 3 minutes to run)
df = extract_transform_data(housing_by_city)

Num of rows: 4500063
Num of cols: 28


In [10]:
# See te first 5 records
df.head()

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,city,state,state_code,property_type,property_type_id,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,inventory,months_of_supply,median_dom,avg_sale_to_list,sold_above_list,parent_metro_region_metro_code,last_updated,period_begin_in_years,period_end_in_years,period_begin_in_months,period_end_in_months
0,2016-09-01,2016-09-30,30,place,6,19944,f,Waukegan,Illinois,IL,Multi-Family (2-4 Unit),4,100000.0,130800.0,47.206204,95.076586,4.0,20.0,5.0,55.0,1.111176,0.250000,29404,2025-03-10 14:20:36,2016,2016,Sep,Sep
1,2019-03-01,2019-03-31,30,place,6,18453,f,Zumbrota,Minnesota,MN,Single Family Residential,6,233500.0,204900.0,142.082386,94.829222,4.0,13.0,3.3,80.0,1.006673,0.250000,39860,2025-03-10 14:20:36,2019,2019,Mar,Mar
2,2015-08-01,2015-08-31,30,place,6,16347,f,Poquoson,Virginia,VA,Single Family Residential,6,345000.0,372450.0,131.699729,139.716868,14.0,79.0,5.6,73.0,0.957002,0.142857,47260,2025-03-10 14:20:36,2015,2015,Aug,Aug
3,2024-01-01,2024-01-31,30,place,6,26370,f,Whitfield,Florida,FL,All Residential,-1,616250.0,129900.0,351.688379,352.348993,4.0,21.0,5.3,17.0,0.979708,0.000000,35840,2025-03-10 14:20:36,2024,2024,Jan,Jan
4,2020-10-01,2020-10-31,30,place,6,7685,f,Hopkins,Minnesota,MN,Single Family Residential,6,330000.0,309450.0,202.949073,177.235722,8.0,19.0,2.4,25.0,1.020724,0.625000,33460,2025-03-10 14:20:36,2020,2020,Oct,Oct


In [ ]:
# Download the transformed dataset as CSV file (not necessary for this purpose)
df.to_csv('Redfin_housing_data_by_City.csv', index=False)